In [0]:
from google.colab import drive,files
drive.mount('/content/drive')   


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
files.upload()

Saving resnet.py to resnet.py


{'resnet.py': b'from __future__ import division\r\n\r\nimport six\r\nfrom keras.models import Model\r\nfrom keras.layers import (\r\n    Input,\r\n    Activation,\r\n    Dense,\r\n    Flatten,\r\n    SeparableConv2D\r\n)\r\nfrom keras.layers.convolutional import (\r\n    Conv2D,\r\n    MaxPooling2D,\r\n    AveragePooling2D,\r\n    ZeroPadding2D\r\n)\r\nfrom keras.layers.pooling import GlobalAveragePooling2D, GlobalMaxPooling2D\r\nfrom keras.layers.merge import add\r\nfrom keras.layers.normalization import BatchNormalization\r\nfrom keras.regularizers import l2\r\nfrom keras import backend as K\r\nfrom keras_applications.imagenet_utils import _obtain_input_shape\r\n\r\n\r\ndef _handle_dim_ordering():\r\n    global ROW_AXIS\r\n    global COL_AXIS\r\n    global CHANNEL_AXIS\r\n    if K.image_dim_ordering() == \'tf\':\r\n        ROW_AXIS = 1\r\n        COL_AXIS = 2\r\n        CHANNEL_AXIS = 3\r\n    else:\r\n        CHANNEL_AXIS = 1\r\n        ROW_AXIS = 2\r\n        COL_AXIS = 3\r\n\r\ncl

In [0]:
!cat resnet.py

from __future__ import division

import six
from keras.models import Model
from keras.layers import (
    Input,
    Activation,
    Dense,
    Flatten,
    SeparableConv2D
)
from keras.layers.convolutional import (
    Conv2D,
    MaxPooling2D,
    AveragePooling2D,
    ZeroPadding2D
)
from keras.layers.pooling import GlobalAveragePooling2D, GlobalMaxPooling2D
from keras.layers.merge import add
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras import backend as K
from keras_applications.imagenet_utils import _obtain_input_shape


def _handle_dim_ordering():
    global ROW_AXIS
    global COL_AXIS
    global CHANNEL_AXIS
    if K.image_dim_ordering() == 'tf':
        ROW_AXIS = 1
        COL_AXIS = 2
        CHANNEL_AXIS = 3
    else:
        CHANNEL_AXIS = 1
        ROW_AXIS = 2
        COL_AXIS = 3

class ResnetBuilder(object):
    @staticmethod
    def identity_conv_block(input,filters,stride=(1,1)):
        identity = input
     

In [0]:
from __future__ import print_function
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils,multi_gpu_model
from keras.callbacks import *
from keras.layers import  Flatten, GlobalAveragePooling2D
from keras.models import load_model
from keras.optimizers import *
from imgaug import augmenters as iaa
from keras import backend as K

import pandas as pd
import numpy as np
import resnet

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
K.tensorflow_backend.set_session(tf.Session(config=config))

Using TensorFlow backend.


In [0]:
!ls "/content/drive/My Drive/dataset"

tiny-imagenet-200.zip		       weights.epochs:008-val_acc:0.503.hdf5
weights.epochs:002-val_acc:0.468.hdf5  weights.epochs:018-val_acc:0.472.hdf5
weights.epochs:002-val_acc:0.484.hdf5  weights.epochs:020-val_acc:0.445.hdf5


In [0]:
#!unzip "/content/drive/My Drive/dataset/tiny-imagenet-200.zip" -d "/content/drive/My Drive/dataset"
!unzip -qq "/content/drive/My Drive/dataset/tiny-imagenet-200.zip"

## Begin with 32x32 training images 

In [0]:
TRAIN_DATA_DIR = 'tiny-imagenet-200/train'
VALIDATION_DATA_DIR = 'tiny-imagenet-200/val'

img_rows = 32
img_cols = 32
img_channels = 3

nb_classes = 200

filepath = "/content/drive/My Drive/dataset/weights.epochs:{epoch:03d}-val_acc:{val_acc:.3f}.hdf5"
  
checkpoint = ModelCheckpoint(filepath,
                            monitor='val_acc',
                            verbose=1,
                            save_best_only=True,
                            mode='max')

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor = np.sqrt(0.1),
                              patience=5,
                              min_lr=0.5e-6,
                              verbose=1)

callbacks_list = [checkpoint,reduce_lr]

In [0]:
train_datagen = ImageDataGenerator(rescale = 1./255)
'''
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=[0.5, 1.30],
                                   brightness_range=[0.5, 1.5],
                                   rotation_range=40)
'''
train_generator = train_datagen.flow_from_directory(
        TRAIN_DATA_DIR, 
        target_size=(img_rows,img_cols),
        color_mode = 'rgb',
        batch_size=256,
        class_mode='categorical',
        shuffle=True,
        seed = 42)


val_data = pd.read_csv('./tiny-imagenet-200/val/val_annotations.txt', sep='\t', header=None, names=['File', 'Class', 'X', 'Y', 'H', 'W'])
val_data.drop(['X', 'Y', 'H', 'W'], axis=1, inplace=True)
val_data.head(3)

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
        val_data, 
        directory='./tiny-imagenet-200/val/images/', 
        x_col='File', y_col='Class', 
        target_size=(32, 32),
        color_mode='rgb', 
        class_mode='categorical',
        batch_size=256, 
        shuffle=True, 
        seed=42)


Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.


## **Model Architecture**

In [0]:
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model = resnet.ResnetBuilder.build_resnet_modified((img_channels,img_rows,img_cols), nb_classes)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 64)   4864        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 64)   0           batch_normalization_1[0][0]      
_____________________________________

In [0]:

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_TEST=validation_generator.n//validation_generator.batch_size

In [0]:
img_rows, img_cols = 32,32

train_datagen = ImageDataGenerator(rescale = 1./255)

'''
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=[0.5, 1.30],
                                   brightness_range=[0.5, 1.5],
                                   rotation_range=35)
'''

train_generator = train_datagen.flow_from_directory(
        TRAIN_DATA_DIR, 
        target_size=(img_rows,img_cols),
        color_mode = 'rgb',
        batch_size=256,
        class_mode='categorical',
        shuffle=True,
        seed = 42)

validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_dataframe(
        val_data, 
        directory='./tiny-imagenet-200/val/images/', 
        x_col='File', y_col='Class', 
        target_size=(img_rows, img_cols),
        color_mode='rgb', 
        class_mode='categorical',
        batch_size=256, 
        shuffle=True, 
        seed=42)

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_TEST=validation_generator.n//validation_generator.batch_size

Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.


### Train on 32x32 dataset  (one session training logs of 30 epochs got overwritten by mistake,as I re-ran the same cell after loading model)

In [0]:
optimizer = Adam(lr= 0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor = 0.2,
                              patience=5,
                              min_lr=0.5e-6,
                              verbose=1)

model = resnet.ResnetBuilder.build_resnet_modified((img_channels,img_rows,img_cols), nb_classes)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

model.load_weights('/content/drive/My Drive/dataset/weights.epochs:009-val_acc:0.414.hdf5')

model.fit_generator(train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_TEST,
                    epochs=20,
                    callbacks = callbacks_list)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
390/390 [==============================] - 493s 1s/step - loss: 3.3693 - acc: 0.2680 - val_loss: 2.9863 - val_acc: 0.3457

Epoch 00001: val_acc improved from -inf to 0.34565, saving model to /content/drive/My Drive/dataset/weights.epochs:001-val_acc:0.346.hdf5
Epoch 2/20
390/390 [==============================] - 481s 1s/step - loss: 2.9629 - acc: 0.3180 - val_loss: 3.1274 - val_acc: 0.3193

Epoch 00002: val_acc did not improve from 0.34565
Epoch 3/20
390/390 [==============================] - 480s 1s/step - loss: 2.8665 - acc: 0.3328 - val_loss: 3.0721 - val_acc: 0.3409

Epoch 00003: val_acc did not improve from 0.34565
Epoch 4/20
390/390 [==============================] - 480s 1s/step - loss: 2.8078 - acc: 0.3479 - val_loss: 3.0578 - val_acc: 0.3367

Epoch 00004: val_acc did not improve from 0.34565
Epoch 5/20
390/390 [==============================] - 48

In [0]:

img_rows, img_cols = 32,32

#train_datagen = ImageDataGenerator(rescale = 1./255)

train_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
        TRAIN_DATA_DIR, 
        target_size=(img_rows,img_cols),
        color_mode = 'rgb',
        batch_size=256,
        class_mode='categorical',
        shuffle=True,
        seed = 42)

validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_dataframe(
        val_data, 
        directory='./tiny-imagenet-200/val/images/', 
        x_col='File', y_col='Class', 
        target_size=(img_rows, img_cols),
        color_mode='rgb', 
        class_mode='categorical',
        batch_size=256, 
        shuffle=True, 
        seed=42)

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_TEST=validation_generator.n//validation_generator.batch_size
model.load_weights('/content/drive/My Drive/dataset/weights.epochs:009-val_acc:0.419.hdf5
model.fit_generator(train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_TEST,
                    epochs=20,
                    callbacks = callbacks_list)

Epoch 1/20
390/390 [==============================] - 482s 1s/step - loss: 2.1617 - acc: 0.4740 - val_loss: 3.0405 - val_acc: 0.3980

Epoch 00001: val_acc did not improve from 0.40599
Epoch 2/20
390/390 [==============================] - 480s 1s/step - loss: 2.1467 - acc: 0.4777 - val_loss: 2.9733 - val_acc: 0.4065

Epoch 00002: val_acc improved from 0.40599 to 0.40651, saving model to /content/drive/My Drive/dataset/weights.epochs:002-val_acc:0.407.hdf5
Epoch 3/20
390/390 [==============================] - 481s 1s/step - loss: 2.1386 - acc: 0.4767 - val_loss: 2.9545 - val_acc: 0.4058

Epoch 00003: val_acc did not improve from 0.40651
Epoch 4/20
390/390 [==============================] - 478s 1s/step - loss: 2.1272 - acc: 0.4800 - val_loss: 3.0324 - val_acc: 0.3982

Epoch 00004: val_acc did not improve from 0.40651
Epoch 5/20
390/390 [==============================] - 478s 1s/step - loss: 2.1221 - acc: 0.4818 - val_loss: 2.9121 - val_acc: 0.4117

Epoch 00005: val_acc improved from 0.40

## **Switch to 64x64 images as now we have validation-accuracy of 41%**

In [0]:
img_rows, img_cols = 64,64

train_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
        TRAIN_DATA_DIR, 
        target_size=(img_rows,img_cols),
        color_mode = 'rgb',
        batch_size=64,
        class_mode='categorical',
        shuffle=True,
        seed = 42)

validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_dataframe(
        val_data, 
        directory='./tiny-imagenet-200/val/images/', 
        x_col='File', y_col='Class', 
        target_size=(img_rows, img_cols),
        color_mode='rgb', 
        class_mode='categorical',
        batch_size=64, 
        shuffle=True, 
        seed=42)

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_TEST=validation_generator.n//validation_generator.batch_size

model = load_model('/content/drive/My Drive/dataset/weights.epochs:009-val_acc:0.414.hdf5')

model.fit_generator(train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_TEST,
                    epochs=10,
                    callbacks = callbacks_list)

Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
1562/1562 [==============================] - 1908s 1s/step - loss: 0.1301 - acc: 0.9605 - val_loss: 3.9809 - val_acc: 0.4734

Epoch 00001: val_acc improved from -inf to 0.47336, saving model to /content/drive/My Drive/dataset/weights.epochs:001-val_acc:0.473.hdf5
Epoch 2/10
1562/1562 [==============================] - 1897s 1s/step - loss: 0.1035 - acc: 0.9674 - val_loss: 4.0993 - val_acc: 0.4729

Epoch 00002: val_acc did not improve from 0.47336
Epoch 3/10
1562/1562 [==============================] - 1893s 1s/step - loss: 0.0946 - acc: 0.9697 - val_loss: 4.2864 - val_acc: 0.4598

Epoch 00003: val_acc did not improve from 0.47336
Epoch 4/10
1562/1562 [==============================] - 1891s 1s/step - loss: 0.0898 - acc: 0.9711 - val_loss: 4.3292 - val_acc: 0.4738

Epo

## **Using Image-Augmentation  as val-acc on 64x64 is now around 50%**

In [0]:
# Using ImgAugmentation now as val-acc is around 50%

def augmentors(batches, flip_value, crop_value, angle):
  seq = iaa.Sequential([
            iaa.Crop(percent=(0,crop_value)),
            iaa.CoarseDropout(p=0.2, size_percent=0.05),
            iaa.Affine(rotate=(-angle, angle)),
            iaa.Multiply((0.4, 1.4)),
            iaa.Sometimes(0.3, iaa.Fliplr(flip_value))]) # 50% flip / horizontal flip of only 30% of the images passed
  while True:
    batch_x, batch_y = next(batches)
    batch_augmented = np.zeros((batch_x.shape[0], batch_x.shape[1], batch_x.shape[2], img_channels)) 
    # NOTE: imgaug works on color images (3 channels). doesn't work on greyscale images with one channel
    batch_augmented = seq.augment_images(batch_x) # calling ImgAug's augmentation on a batch of images
    yield (batch_augmented, batch_y) 


In [0]:
# CLR
class CyclicLR(Callback):
    "source: https://github.com/bckenstler/CLR/blob/master/clr_callback.py"
    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())

In [0]:
# Using ImgAugmentation now as val-acc is around 50%

img_rows, img_cols = 64,64

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=[0.5, 1.30],
                                   brightness_range=[0.5, 1.5],
                                   rotation_range=35)

train_generator = train_datagen.flow_from_directory(
        TRAIN_DATA_DIR, 
        target_size=(img_rows,img_cols),
        color_mode = 'rgb',
        batch_size=64,
        class_mode='categorical',
        shuffle=True,
        seed = 42)

validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_dataframe(
        val_data, 
        directory='./tiny-imagenet-200/val/images/', 
        x_col='File', y_col='Class', 
        target_size=(img_rows, img_cols),
        color_mode='rgb', 
        class_mode='categorical',
        batch_size=64, 
        shuffle=True, 
        seed=42)

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_TEST=validation_generator.n//validation_generator.batch_size
print("STEP_SIZE_TRAIN", STEP_SIZE_TRAIN)


Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.
STEP_SIZE_TRAIN 1562


### Using Cyclic Learning Rate from Keras (Policy: Triangular)

In [0]:
#Instantiate CLR (Policy = Triangular , Now with step size = 2*Iterations)
clr = CyclicLR(base_lr=0.001, max_lr=0.006,step_size=3162)
callbacks_list = [checkpoint,clr]


#Load back checkpointed model of 50% val accuracy
model = load_model('/content/drive/My Drive/dataset/weights.epochs:008-val_acc:0.503.hdf5')

model.fit_generator(train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_TEST,
                    epochs=30,
                    callbacks = callbacks_list)

Instructions for updating:
Use tf.cast instead.
Epoch 1/30
1562/1562 [==============================] - 1933s 1s/step - loss: 3.5663 - acc: 0.2305 - val_loss: 4.3746 - val_acc: 0.2202

Epoch 00001: val_acc improved from -inf to 0.22015, saving model to /content/drive/My Drive/dataset/weights.epochs:001-val_acc:0.220.hdf5
Epoch 2/30
1562/1562 [==============================] - 1924s 1s/step - loss: 3.3445 - acc: 0.2524 - val_loss: 4.2517 - val_acc: 0.1348

Epoch 00002: val_acc did not improve from 0.22015
Epoch 3/30
1562/1562 [==============================] - 1921s 1s/step - loss: 3.2406 - acc: 0.2726 - val_loss: 4.1710 - val_acc: 0.1829

Epoch 00003: val_acc did not improve from 0.22015
Epoch 4/30
1562/1562 [==============================] - 1921s 1s/step - loss: 2.8194 - acc: 0.3479 - val_loss: 2.9567 - val_acc: 0.3679

Epoch 00004: val_acc improved from 0.22015 to 0.36785, saving model to /content/drive/My Drive/dataset/weights.epochs:004-val_acc:0.368.hdf5
Epoch 5/30
1562/1562 [===

In [0]:
#Instantiate CLR (Policy = Traingular)
clr = CyclicLR(base_lr=0.001, max_lr=0.006,step_size=1562)
callbacks_list = [checkpoint,clr]

### Colab disconnected resuming below

In [0]:
model = load_model('/content/drive/My Drive/dataset/weights.epochs:020-val_acc:0.445.hdf5')

model.fit_generator(train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_TEST,
                    epochs=20,
                    callbacks = callbacks_list)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
1562/1562 [==============================] - 1883s 1s/step - loss: 2.1890 - acc: 0.4720 - val_loss: 3.6805 - val_acc: 0.3230

Epoch 00001: val_acc improved from -inf to 0.32302, saving model to /content/drive/My Drive/dataset/weights.epochs:001-val_acc:0.323.hdf5
Epoch 2/20
1562/1562 [==============================] - 1870s 1s/step - loss: 2.2335 - acc: 0.4627 - val_loss: 2.4068 - val_acc: 0.4841

Epoch 00002: val_acc improved from 0.32302 to 0.48410, saving model to /content/drive/My Drive/dataset/weights.epochs:002-val_acc:0.484.hdf5
Epoch 3/20
 330/1562 [=====>........................] - ETA: 23:46 - loss: 1.9282 - acc: 0.5250

### Colab again disconnected resuming below

In [0]:

model = load_model('/content/drive/My Drive/dataset/weights.epochs:002-val_acc:0.484.hdf5')

model.fit_generator(train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_TEST,
                    epochs=20,
                    callbacks = callbacks_list)

Epoch 1/20
1562/1562 [==============================] - 1909s 1s/step - loss: 1.9767 - acc: 0.5147 - val_loss: 4.1197 - val_acc: 0.3182

Epoch 00001: val_acc improved from -inf to 0.31821, saving model to /content/drive/My Drive/dataset/weights.epochs:001-val_acc:0.318.hdf5
Epoch 2/20
1562/1562 [==============================] - 1897s 1s/step - loss: 2.0574 - acc: 0.4974 - val_loss: 2.4356 - val_acc: 0.4837

Epoch 00002: val_acc improved from 0.31821 to 0.48370, saving model to /content/drive/My Drive/dataset/weights.epochs:002-val_acc:0.484.hdf5
Epoch 3/20
1562/1562 [==============================] - 1896s 1s/step - loss: 2.1185 - acc: 0.4859 - val_loss: 3.4495 - val_acc: 0.3316

Epoch 00003: val_acc did not improve from 0.48370
Epoch 4/20
1562/1562 [==============================] - 1896s 1s/step - loss: 2.1555 - acc: 0.4785 - val_loss: 2.7192 - val_acc: 0.4638

Epoch 00004: val_acc did not improve from 0.48370
Epoch 5/20
1562/1562 [==============================] - 1896s 1s/step - l

### Above cell-ouput shows max validation accuracy acheived so far is 50.382%